In [ ]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
/content/0001.png

In [ ]:
import pandas as pd
df=pd.read_csv('/content/flower_labels.csv')

In [ ]:
df.head()

,file,label
0,0001.png,0
1,0002.png,0
2,0003.png,2
3,0004.png,0
4,0005.png,0


In [ ]:
df = df.replace({0:'phlox',1:'rose',2:'calendula',3:'iris',4:'leucanthemum maximum',
                 5:'bellflower',6:'viola',7:'rudbeckia laciniata',
                 8:'peony',9:'aquilegia'})

In [ ]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np

import cv2
SEED = 42
EPOCHS = 50
BATCH_SIZE = 32 
img_size = 224
train_df, test_df = train_test_split(df, 
                                     test_size=0.5, 
                                     random_state=SEED, 
                                     stratify=df['label'].values)



def create_datasets(df, img_size):
    imgs = []
    for file in tqdm(df['file']):
        img = cv2.imread('/content/'+file)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (img_size,img_size))
        imgs.append(img)
    # not normalize    
    imgs = np.array(imgs)
    df = pd.get_dummies(df['label'])
    return imgs, df


train_imgs, train_df = create_datasets(train_df, img_size)
test_imgs, test_df = create_datasets(test_df, img_size)

100%|██████████| 105/105 [00:00<00:00, 736.19it/s]


In [ ]:
img_size = [224, 224]
mobilnet = VGG19(input_shape=img_size + [3], weights='imagenet', include_top=False)

80142336/80134624 [==============================] - 1s 0us/step


In [ ]:
from tensorflow.keras.layers import *
from tensorflow.keras.applications.vgg19 import VGG19
num_classes = len(df.label.value_counts())
img_size= 224
import tensorflow as tf
def build_model(VGG19, img_size, n):
    inp = Input(shape=(img_size,img_size, n))
    mobilnet = VGG19(input_shape=(img_size,img_size,n),
                    weights='imagenet',
                    include_top=False)
    # freeze ResNet
    mobilnet.trainable = False
    x = mobilnet(inp)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x =Dropout(0.5)(x)
    x = Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs=inp, outputs=x)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


vgg19 = build_model(VGG19, img_size, 3)
vgg19.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg19 (Functional)           (None, 7, 7, 512)         20024384  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 20,029,514
Trainable params: 5,130
Non-trainable params: 20,024,384
_________________________________________________________________


In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('vgg19.h5', 
                                                monitor='loss', 
                                                save_best_only=True,
                                                save_weights_only=True)
vgg19.fit(train_imgs, train_df, batch_size=BATCH_SIZE,
          epochs=30, verbose=0, callbacks=[checkpoint])
vgg19.load_weights('vgg19.h5')


vgg19.evaluate(test_imgs, test_df)

4/4 [==============================] - 45s 11s/step - loss: 0.7991 - accuracy: 0.7238


[0.7991319298744202, 0.723809540271759]